In [64]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

station_popularity = pd.read_csv('data/original_data/MTA_Subway_Stations.csv')
station_location = pd.read_excel('data\original_data\mta_subway_tables.xlsx', sheet_name='Subway Rank')

<>:6: SyntaxWarning: invalid escape sequence '\o'
<>:6: SyntaxWarning: invalid escape sequence '\o'
C:\Users\david\AppData\Local\Temp\ipykernel_8880\2412086370.py:6: SyntaxWarning: invalid escape sequence '\o'
  station_location = pd.read_excel('data\original_data\mta_subway_tables.xlsx', sheet_name='Subway Rank')


In [65]:
station_popularity.head()

,GTFS Stop ID,Station ID,Complex ID,Division,Line,Stop Name,Borough,CBD,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Northbound,ADA Southbound,ADA Notes,Georeference
0,637,408,619,IRT,Lexington Av,Bleecker St,M,True,6,Subway,40.725915,-73.994659,Uptown,Downtown,1,1,1,NaN,POINT (-73.994659 40.725915)
1,B23,70,70,BMT,West End,Bay 50 St,Bk,False,D,Elevated,40.588841,-73.983765,Manhattan,Coney Island,0,0,0,NaN,POINT (-73.983765 40.588841)
2,607,365,365,IRT,Pelham,Castle Hill Av,Bx,False,6,Elevated,40.834255,-73.851222,Pelham Bay,Manhattan,0,0,0,NaN,POINT (-73.851222 40.834255)
3,S01,139,627,BMT,Franklin Shuttle,Franklin Av,Bk,False,S,Elevated,40.680596,-73.955827,Last Stop,Prospect Park,1,1,1,NaN,POINT (-73.955827 40.680596)
4,M18,102,625,BMT,Jamaica,Delancey St-Essex St,M,True,M J Z,Subway,40.718315,-73.987437,Brooklyn,Inbound,0,0,0,NaN,POINT (-73.987437 40.718315)


In [66]:
station_location.head()

,CCYY,STAID,STATION,SEGMENT,SECTOR,BOROUGH,AVGWKDRIDER,AVGSATRIDER,AVGSUNRIDER,AVGWKND,ANN_RID,Wkdy Rank,Wknd Rank,Annual Rank,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,2023.0,10,"Wall St (2,3)",Broadway/7 Av (CBD),Downtown,Manhattan,12395,5956,4832,10788,3729954,76.0,112.0,82.0,NaN,1.0,NaN,1994097.0
1,2023.0,50,Rector St (1),Broadway/7 Av (CBD),Downtown,Manhattan,4898,3770,2850,6620,1606245,209.0,181.0,202.0,NaN,1.0,NaN,NaN
2,2023.0,60,WTC Cortlandt (1),Broadway/7 Av (CBD),Downtown,Manhattan,10218,8069,5810,13879,3358064,98.0,82.0,95.0,NaN,1.0,NaN,NaN
3,2023.0,70,"Chambers St (1,2,3)",Broadway/7 Av (CBD),Downtown,Manhattan,12798,5651,4283,9934,3782605,70.0,130.0,77.0,NaN,1.0,NaN,NaN
4,2023.0,80,Franklin St (1),Broadway/7 Av (CBD),Valley,Manhattan,4337,2400,1822,4222,1329726,227.0,248.0,234.0,NaN,1.0,NaN,NaN


In [67]:
station_popularity = station_popularity[['Stop Name', 'Georeference']]
station_location = station_location[['STATION', 'BOROUGH', 'ANN_RID']]
station_location.rename(columns={'STATION':'Stop Name', 'ANN_RID':'Annual Riders'}, inplace=True)

In [68]:
def remove_last_str(x):
    y = (str(x)).split('(')
    return y[0].strip()
station_location['Stop Name'] = station_location['Stop Name'].apply(remove_last_str)

In [69]:
def clean_name(x):
    x=x.replace('-', ' ')
    x=x.upper()
    x=x.strip()
    return x
station_location['Stop Name'] = station_location['Stop Name'].apply(clean_name)
station_popularity['Stop Name'] = station_popularity['Stop Name'].apply(clean_name)

In [70]:
station_info = pd.merge(station_location, station_popularity, on=['Stop Name'])
station_info.drop_duplicates(subset='Stop Name', inplace=True)
station_info.tail()

,Stop Name,BOROUGH,Annual Riders,Georeference
633,MYRTLE WYCKOFF AVS,Brooklyn,5009020,POINT (-73.911586 40.699814)
637,CONEY ISLAND STILLWELL AV,Brooklyn,3220518,POINT (-73.981233 40.577422)
638,QUEENSBORO PLAZA,Queens,3573523,POINT (-73.940202 40.750582)
640,COURT SQ,Queens,5381184,POINT (-73.943832 40.746554)
642,SUTPHIN BLVD ARCHER AV JFK AIRPORT,Queens,5941974,POINT (-73.807969 40.700486)


In [71]:
def point_strip(x):
    x = str(x)
    y = x[7:-1]
    return y
station_info['Georeference'] = station_info['Georeference'].apply(point_strip)
station_info.dropna(subset='Georeference', inplace=True)
station_info[['Station Latitude','Station Longitude']] = station_info['Georeference'].apply(lambda x: pd.Series(str(x).split(" ")))
station_info.drop(columns=['Georeference'], inplace=True)
station_info['Station Latitude'] = station_info['Station Latitude'].astype('float')
station_info['Station Longitude'] = station_info['Station Longitude'].astype('float')
station_info.tail()

,Stop Name,BOROUGH,Annual Riders,Station Latitude,Station Longitude
633,MYRTLE WYCKOFF AVS,Brooklyn,5009020,-73.911586,40.699814
637,CONEY ISLAND STILLWELL AV,Brooklyn,3220518,-73.981233,40.577422
638,QUEENSBORO PLAZA,Queens,3573523,-73.940202,40.750582
640,COURT SQ,Queens,5381184,-73.943832,40.746554
642,SUTPHIN BLVD ARCHER AV JFK AIRPORT,Queens,5941974,-73.807969,40.700486


In [72]:
station_info.to_csv('data/cleaned_data/MTA_Subway_Info.csv', index=False)